In [10]:
import pyreadr
transactions_df = pyreadr.read_r("/home/spadef/data/IDEA_DeFi_Research/Data/Lending_Protocols/Aave/V3/Polygon/transactionsFinSurvival.rds")[None]
# transactions_df.to_csv("/home/spadef/data/IDEA_DeFi_Research/Data/Lending_Protocols/Aave/V3/Polygon/transactionsFinSurvival.csv", index=False)
print(transactions_df["timestamp"].max())

1755726959.0


In [12]:
import os
import pandas as pd

EVENTS = ["Deposit", "Withdraw", "Repay", "Borrow", "Liquidated"]
DATA_PATH = "./data/"

In [ ]:
users = {}
for index_event in EVENTS:
    for outcome_event in EVENTS:
        csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
        if not os.path.exists(csv_path):
            continue
        df = pd.read_csv(csv_path)
        for _, row in df.iterrows():
            if row["user"] not in users:
                users[row["user"]] = {
                    "user_address": "0xUser123",
                    "description": "My custom user profile",
                    "transactions": [],
                }
            users[row["user"]]["transactions"].append(
                {"action": row[""], "symbol": "USDC", "amount": 5000, "timestamp": 0}
            )

In [4]:
for index_event in EVENTS:
    for outcome_event in EVENTS:
        if not (index_event == "Deposit" and outcome_event == "Withdraw"):
            continue
        csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
        if not os.path.exists(csv_path):
            continue
        df = pd.read_csv(csv_path)
        for col in ["Index Event", "Outcome Event"]:
            if col in df.columns:
                mask = df[col].notna()
                df.loc[mask, col] = (
                    df.loc[mask, col]
                    .astype(str)
                    .str.replace("account liquidated", "liquidated", case=False, regex=False)
                    .str.strip()
                )
        df.to_csv(csv_path, index=False)
        print(f"Updated {csv_path}")

Updated ./data/Deposit/Withdraw/data.csv


In [5]:
for index_event in EVENTS:
    for outcome_event in EVENTS:
        if not (index_event == "Deposit" and outcome_event == "Withdraw"):
            continue
        csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
        if not os.path.exists(csv_path):
            continue
        df = pd.read_csv(csv_path)
        for col in ["Index Event", "Outcome Event"]:
            if col in df.columns:
                mask = df[col].notna()
                df.loc[mask, col] = (
                    df.loc[mask, col]
                    .astype(str)
                    .str.replace("liquidation", "liquidated", case=False, regex=False)
                    .str.strip()
                )
        df.to_csv(csv_path, index=False)
        print(f"Updated {csv_path}")

Updated ./data/Deposit/Withdraw/data.csv


In [ ]:
for event in EVENTS:
    csv_path = os.path.join(DATA_PATH, event, event, "data.csv")
    if not (event == "Deposit" and event == "Withdraw"):
        continue
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    for col in ["Outcome Event"]:
        if col in df.columns:
            mask = df[col].notna()
            df.loc[mask, col] = (
                df.loc[mask, col]
                .astype(str)
                .str.replace(event + "_outcome", event, case=False, regex=False)
                .str.strip()
            )
    df.to_csv(csv_path, index=False)
    print(f"Updated {csv_path}")

In [9]:
for event in EVENTS:
    csv_path = os.path.join(DATA_PATH, event, event, "data.csv")
    if not (event == "Deposit" and event == "Withdraw"):
        continue
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    for col in ["Outcome Event"]:
        if col in df.columns:
            mask = df[col].notna()
            df.loc[mask, col] = (
                df.loc[mask, col]
                .astype(str)
                .str.replace(event, event.lower(), case=False, regex=False)
                .str.strip()
            )
    df.to_csv(csv_path, index=False)
    print(f"Updated {csv_path}")

In [ ]:
index_event = "Liquidated"
for outcome_event in EVENTS:
    csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    for col in ["Index Event", "Outcome Event"]:
        if col in df.columns:
            mask = df[col].notna()
            df.loc[mask, col] = (
                df.loc[mask, col]
                .astype(str)
                .str.replace("liquidation", "liquidated", case=False, regex=False)
                .str.strip()
            )
    df.to_csv(csv_path, index=False)
    print(f"Updated {csv_path}")

Updated ./data/Liquidated/Deposit/data.csv
Updated ./data/Liquidated/Withdraw/data.csv
Updated ./data/Liquidated/Repay/data.csv
Updated ./data/Liquidated/Borrow/data.csv
Updated ./data/Liquidated/Liquidated/data.csv


In [ ]:
import json
from collections import defaultdict

usersToTest = [
    "0xae0cc724fc85cd394435eb4f9ddfd3a8c088b4e2",
    "0x0e7b3aea0428137786edf20617670179435c530b",
    "0x97dc47d17ab56c32a4d2da1839e3ff261304e95d",
    "0x5091d57b4feb57db1da04b51cc5454f371b623a6",
]

for user in usersToTest:
    with open(
        f"../data/IDEA_DeFi_Research/Data/CSV/profile-generation/user_{user}.json"
    ) as f:
        userTest = json.load(f)

    userTransactions = userTest["transactions"]
    print(next(
            (e for i, e in enumerate(userTransactions) if (e["action"] == "Liquidated"))
        )['debt_symbol'])
    userTransactions = userTransactions[
        : next(
            (i for i, e in enumerate(userTransactions) if (e["action"] == "Liquidated")), -1
        )
    ]
    borrows = defaultdict(int)
    repays = defaultdict(int)
    for transaction in userTransactions:
        if transaction["action"] == "Borrow":
            borrows[transaction["symbol"]] += transaction["amount"]
        elif transaction["action"] == "Repay":
            repays[transaction["symbol"]] += transaction["amount"]
    print(f'{user}\'s "Debt" before liquidation: {[f'{symbol}: {borrows[symbol] - repays[symbol]}' for symbol in borrows.keys()]}')

WETH
0xae0cc724fc85cd394435eb4f9ddfd3a8c088b4e2's"Debt" before liquidation: ['USDC: 898.7291915312042', 'WETH: -1.6900000000003718e-06']
USDC
0x0e7b3aea0428137786edf20617670179435c530b's"Debt" before liquidation: ['USDC: 0.0', 'WETH: 75.7625662202381']
USDC
0x97dc47d17ab56c32a4d2da1839e3ff261304e95d's"Debt" before liquidation: ['USDC: 7.854323']
WETH
0x5091d57b4feb57db1da04b51cc5454f371b623a6's"Debt" before liquidation: ['USDC: 0.0']


In [ ]:
import pandas as pd

with open('../data/IDEA_DeFi_Research/Data/Lending_Protocols/Aave/V2/Polygon/users.csv') as f:
    old_users = pd.read_csv(f)['id']
old_users.head()

0    0x0000000000000000000000000000000000000001
1    0x0000000000000000000000000000000000001010
2    0x000000000000000000000000000000000000dead
3    0x0000000000085a12481aedb59eb3200332aca597
4    0x00000000000a29a0800f6f557ddbbe8249397de7
Name: id, dtype: object

In [10]:
with open('./data/users.csv') as f:
    users = pd.read_csv(f)['id']
new_users = set(users) - set(old_users)
print(f'Out of {len(users)} users, {len(new_users)} users were not among the {len(old_users)} old users.')

Out of 365677 users, 308205 users were not among the 1121518 old users.


In [ ]:
usersWithLiquidation = set()
outcome_event = "Liquidated"
for index_event in EVENTS:
    if index_event == outcome_event:
        continue
    csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    usersWithLiquidation = usersWithLiquidation.union(set(df['user']))
    print(f"Got liquidated users from {csv_path}")
new_liquidated_users = set(usersWithLiquidation) - set(old_users)
print(f'Out of {len(usersWithLiquidation)} liquidated users, {len(new_liquidated_users)} liquidated users were not among the {len(old_users)} old users.')

Got liquidated users from ./data/Deposit/Liquidated/data.csv
Got liquidated users from ./data/Withdraw/Liquidated/data.csv
Got liquidated users from ./data/Repay/Liquidated/data.csv
Got liquidated users from ./data/Borrow/Liquidated/data.csv
Out of 259775 liquidated users, 204309 liquidated users were not among the 1121518 old users.


In [ ]:
usersWithLiquidation = set()
index_event = "Liquidated"
for outcome_event in EVENTS:
    if index_event == outcome_event:
        continue
    csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    usersWithLiquidation = usersWithLiquidation.union(set(df['user']))
    print(f"Got liquidated users from {csv_path}")
new_liquidated_users = set(usersWithLiquidation) - set(old_users)
print(f'Out of {len(usersWithLiquidation)} liquidated users, {len(new_liquidated_users)} liquidated users were not among the {len(old_users)} old users.')

Got liquidated users from ./data/Liquidated/Deposit/data.csv
Got liquidated users from ./data/Liquidated/Withdraw/data.csv
Got liquidated users from ./data/Liquidated/Repay/data.csv
Got liquidated users from ./data/Liquidated/Borrow/data.csv
Out of 13743 liquidated users, 8899 liquidated users were not among the 1121518 old users.


In [16]:
[user for user in usersToTest if user in new_liquidated_users]

['0x97dc47d17ab56c32a4d2da1839e3ff261304e95d',
 '0x5091d57b4feb57db1da04b51cc5454f371b623a6']

In [17]:
import json

with open('./Aave-Simulator/results/zero-debt-liquidation-analysis/analysis_20251218_164135.json') as f:
    zero_debt_liquidated_users = [case['user_address'] for case in json.load(f)['zero_debt_cases']]
new_zero_liquidated_users = set(zero_debt_liquidated_users) - set(old_users)
print(f'Out of {len(zero_debt_liquidated_users)} zero-debt liquidated users, {len(new_zero_liquidated_users)} zero-debt liquidated users were not among the {len(old_users)} old users.')

Out of 1328 zero-debt liquidated users, 366 zero-debt liquidated users were not among the 1121518 old users.


In [1]:
from actionAgentTraining import verify_amount_feature_effect, get_model_for_pair_and_date
import pandas as pd

model = get_model_for_pair_and_date("Repay", "Liquidated", pd.to_datetime("2024-05-05 15:33:51.800000"))
print(model)

KeyboardInterrupt: 